## Header to define `Movie` struct

In [ ]:
%%writefile movie.h
#ifndef MOVIE_H
#define MOVIE_H

struct Movie
{
    int id, duration, category, startTime, endTime;
};

#endif

Overwriting movie.h


## Cuda utils with helper functions

In [ ]:
%%writefile cuda_utils.h
#ifndef CUDA_UTILS_H
#define CUDA_UTILS_H

#include <thrust/device_vector.h>
#include <vector>

using namespace std;

int fillDataVectors(vector<int> &catLimit, vector<int> &start, vector<int> &end, vector<int> &cat);
thrust::device_vector<bool> convertToBinaryVec(int n, int size);

#endif

Overwriting cuda_utils.h


In [ ]:
%%writefile cuda_utils.cu
#include <algorithm>
#include <thrust/device_vector.h>
#include <iostream>
#include "cuda_utils.h"
#include "movie.h"

using namespace std;

int fillDataVectors(vector<int> &catLimit, vector<int> &start, vector<int> &end, vector<int> &cat)
{
    int totalMovies;
    int totalCategories;

    cin >> totalMovies >> totalCategories;

    // Fill category limit vector
    catLimit.reserve(totalCategories);
    int categoryLimit;
    for (int i = 0; i < totalCategories; i++)
    {
        cin >> categoryLimit;
        catLimit.push_back(categoryLimit);
    }

    // Fill start, end and cat vectors
    int startValue, endValue, catValue;
    vector<Movie> movies;
    int movieCount = 0;
    for (int i = 0; i < totalMovies; i++)
    {
        cin >> startValue >> endValue >> catValue;
        if (startValue >= endValue)
            continue;

        movies.push_back({i, endValue - startValue, catValue, startValue, endValue});
        movieCount++;
    }

    sort(movies.begin(), movies.end(), [](const Movie &lhs, const Movie &rhs)
         { return lhs.startTime < rhs.startTime; });

    for (auto movie : movies)
    {
        start.push_back(movie.startTime);
        end.push_back(movie.endTime);
        cat.push_back(movie.category);
    }

    return movieCount;
}

thrust::device_vector<bool> convertToBinaryVec(int n, int size)
{

    thrust::device_vector<bool> binaryVec(size);
    for (int i = 0; i < n; i++)
    {
        binaryVec[i] = n & (1 << i);
    }
    return binaryVec;
}

Overwriting cuda_utils.cu


## Cuda code (busca exaustiva em GPU)

In [ ]:
%%writefile main.cu

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/iterator/counting_iterator.h>
#include <thrust/functional.h>
#include <thrust/count.h>
#include <iostream>
#include <math.h>
#include "cuda_utils.h"

using namespace std;

struct validate
{
    const int movieCount;
    thrust::device_ptr<const int> catLimit;
    thrust::device_ptr<const int> movieStart;
    thrust::device_ptr<const int> movieEnd;
    thrust::device_ptr<const int> movieCat;

    validate(
        const int movieCount_,
        thrust::device_ptr<const int> catLimit_,
        thrust::device_ptr<const int> movieStart_,
        thrust::device_ptr<const int> movieEnd_,
        thrust::device_ptr<const int> movieCat_)
        : movieCount(movieCount_),
          catLimit(catLimit_),
          movieStart(movieStart_),
          movieEnd(movieEnd_),
          movieCat(movieCat_){};

    __host__ __device__ int operator()(const int &bin)
    {
        int lastIndex = -1;
        int result = 0;
        int localCatArr[20] = {0};

        for (int i = 0; i < movieCount; i++)
        {
            bool bit = (bin & (1 << i)) != 0;
            if (!bit)
                continue;

            if (lastIndex == -1)
            {
                lastIndex = i;
                result++;
                continue;
            }

            if (localCatArr[movieCat[i] - 1] == catLimit[movieCat[i] - 1])
            {
                return 0;
            }

            if (movieEnd[lastIndex] > movieStart[i])
            {
                return 0;
            }

            localCatArr[movieCat[i] - 1]++;
            lastIndex = i;
            result++;
        }

        return result;
    }
};

int main()
{
    vector<int> catLimit;
    vector<int> movieStart;
    vector<int> movieEnd;
    vector<int> movieCat;

    int movieCount = fillDataVectors(catLimit, movieStart, movieEnd, movieCat);
    thrust::device_vector<int> devCatLimit(catLimit);
    thrust::device_vector<int> devStart(movieStart);
    thrust::device_vector<int> devEnd(movieEnd);
    thrust::device_vector<int> devCat(movieCat);

    thrust::counting_iterator<int> startBin(1);
    thrust::counting_iterator<int> endBin = startBin + (int)pow(2, movieCount) - 1;

    validate validateFunctor(movieCount, devCatLimit.data(), devStart.data(), devEnd.data(), devCat.data());

    thrust::device_vector<int> results(endBin - startBin);
    thrust::transform(startBin, endBin, results.begin(), validateFunctor);
    

    int maxMovies = thrust::reduce(results.begin(), results.end(), 0, thrust::maximum<int>());

    cout << (int)pow(2, movieCount) - 1 << " " <<maxMovies;

    return 0;
}
    

Overwriting main.cu


## Compila o código cuda

In [ ]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o main main.cu cuda_utils.cu

## Utils e helper para código da busca exaustiva OMP

In [ ]:
%%writefile utils.h
#ifndef UTILS_H
#define UTILS_H

#include <tuple>
#include <vector>
#include "movie.h"

using namespace std;

tuple<int, int> getInputParameters();
vector<int> setCategoryVector(int totalCategories);
vector<Movie> setMovieVector(int totalMovies);

#endif

Overwriting utils.h


In [ ]:
%%writefile utils.cpp
#include <iostream>
#include "utils.h"

using namespace std;

tuple<int, int> getInputParameters()
{
    int totalMovies;
    int totalCategories;

    cin >> totalMovies >> totalCategories;
    //cout << totalMovies << " " << totalCategories << endl;

    return make_tuple(totalMovies, totalCategories);
}

vector<int> setCategoryVector(int totalCategories)
{
    vector<int> categoryVector;
    categoryVector.reserve(totalCategories);

    int categoryLimit;
    for (int i = 0; i < totalCategories; i++)
    {
        cin >> categoryLimit;
        //cout << "cat " << i << "= " << categoryLimit << endl;
        categoryVector.push_back(categoryLimit);
    }

    return categoryVector;
}

vector<Movie> setMovieVector(int totalMovies)
{
    vector<Movie> movieVector;
    movieVector.reserve(totalMovies);

    int id, duration, category, startTime, endTime;

    int count = 0;
    for (int i = 0; i < totalMovies; i++)
    {
        cin >> startTime >> endTime >> category;
        //cout << "movie " << i << "->" << startTime << " " << endTime << " " << category << endl;
        id = count;
        duration = endTime - startTime;

        if (duration <= 0)
            continue;

        movieVector.push_back({id, duration, category, startTime, endTime});
    }
    return movieVector;
}

Overwriting utils.cpp


## Código da busca exaustiva OMP para comparação com GPU

In [ ]:
%%writefile exhaustive_omp.cpp

#include <algorithm>
#include <iostream>
#include <math.h>
#include <omp.h>
#include "movie.h"
#include "utils.h"

using namespace std;

int validateInput(int combinationCode, vector<Movie> movies, vector<int> categoryLimit, int movieCount);

int main()
{
    // Limita número de threads para 2, possui colab só fornece 2 cores.
    // Isso tenta otimizar o código para o colab
    omp_set_num_threads(2);
 
    tuple<int, int> output = getInputParameters();
    int totalMovies = get<0>(output);
    int totalCategories = get<1>(output);
 
    vector<int> categoryLimit = setCategoryVector(totalCategories);
    vector<Movie> movies = setMovieVector(totalMovies);
    int movieCount = movies.size();

    // Define last binary for movie combinations
    int maxBin = (int)pow(2, movieCount) - 1;

    // sort movies by increasing end time
    sort(movies.begin(), movies.end(), [](Movie &i, Movie &j)
         { return i.endTime < j.endTime; });

    int maxMovies = 0;

#pragma omp parallel for
    for (int i = 1; i <= maxBin; i++)
    {
        if (__builtin_popcount(i) > 24)
            continue;

        int combinationMovies = validateInput(i, movies, categoryLimit, movieCount);

#pragma omp critical
        {
            if (combinationMovies > maxMovies)
                maxMovies = combinationMovies;
        }
    }

    cout << maxBin << " " << maxMovies;

    return 0;
}

int validateInput(int combinationCode, vector<Movie> dataVec, vector<int> catLimit, int movieCount)
{
    int lastIndex = -1;
    int result = 0;

    for (int i = 0; i < movieCount; i++)
    {
        bool bit = (combinationCode & (1 << i)) != 0;
        if (!bit)
            continue;

        if (lastIndex == -1)
        {
            lastIndex = i;
            result++;
            continue;
        }

        if (catLimit[dataVec[i].category - 1] == 0)
            return 0;
        if (dataVec[lastIndex].endTime > dataVec[i].startTime)
            return 0;

        catLimit[dataVec[i].category - 1]--;
        lastIndex = i;
        result++;
    }

    return result;
}

Overwriting exhaustive_omp.cpp


## Compila código da busca exaustiva OMP (

In [ ]:
!g++ -g -Wall -fopenmp -o exhaustive_omp ./exhaustive_omp.cpp ./utils.cpp

## Gera inputs para teste, localmente, usando script de python

In [ ]:
import random
import datetime

def generate_tests(n, m):
    seed = datetime.datetime.now().timestamp()
    generator = random.Random(seed)

    distribution_dif = lambda: random.normalvariate(3, 1.0)

    cat_limit = lambda: random.randint(1, 10)
    distribution_hr = lambda: random.randint(0, 23)
    distribution_cat = lambda: random.randint(1, m)

    cat_limit_list = []
    for i in range(m):
        cat_limit_list.append(cat_limit())

    movie_list = []
    for i in range(n):
        hora_inicio = distribution_hr()
        dif_media = distribution_dif()
        hora_fim = (hora_inicio + round(dif_media)) % 24
        categoria = distribution_cat()
        movie_list.append((hora_inicio, hora_fim, categoria))

    return cat_limit_list, movie_list

In [ ]:
test = generate_tests(5, 2)
a = lambda x: str(x[0]) + " " + str(x[1]) + " " + str(x[2])

print(test[1])
a(test[1][0])

[(13, 16, 1), (4, 7, 2), (4, 5, 1), (20, 1, 1), (7, 11, 2)]


'13 16 1'

In [ ]:
def parse_input(cat_limit_list, movie_list):
    input_str = str(len(movie_list)) + " " + str(len(cat_limit_list)) + " "

    for i in cat_limit_list:
      input_str += str(i) + " "

    for i in movie_list:
      input_str += str(i) + " "

    return input_str.strip().replace('(', '').replace(')', '').replace(',', '')

test = generate_tests(5, 2)
parse_input(test[0], test[1])

'5 2 3 3 18 23 1 2 7 1 22 1 1 20 0 1 10 12 2'

In [ ]:
import os
import subprocess

test = generate_tests(5,2)
print(test)
input_str = parse_input(test[0], test[1])

proc = subprocess.run(["./exhaustive_omp"], input=input_str, text=True, capture_output=True)

print(proc.stdout, proc.stderr)

([8, 5], [(16, 18, 1), (5, 9, 1), (10, 14, 2), (15, 18, 2), (3, 6, 2)])
31 3 


## Gera inputs de teste

In [ ]:
test_list = []

for cat_count in range(1, 20):
  for movie_count in range(1, 31):
    test_list.append(generate_tests(movie_count, cat_count))

In [ ]:
print(len(test_list))
print(test_list[0])

570
([1], [(3, 5, 1)])


## Benchmark OMP

In [ ]:
import subprocess
import time

def benchmark_input(input_obj) -> tuple[str, str, float, int, int]:
    input_str = parse_input(input_obj[0], input_obj[1])
    start = time.perf_counter()
    proc = subprocess.run(
        ["./exhaustive_omp"], input=input_str, text=True, capture_output=True
        )
    end = time.perf_counter()

    return proc.stdout, proc.stderr, end - start, len(input_obj[0]), len(input_obj[1])

In [ ]:
omp_result = {}

counter = 0
for i in test_list:
    if (len(i[1]) > 26): continue
    counter += 1
    if counter%50 == 0: print(counter)

    test_output = benchmark_input(i)
    key = str(test_output[-1]) + " " + str(test_output[-2])    
              
    omp_result[key] = test_output

50
100
150
200
250
300
350
400
450


In [ ]:
print(omp_result['26 5'])

('8388607 7', '', 2.639602013000058, 5, 26)


## Benchmark GPU

In [ ]:
def benchmark_input_gpu(input_obj) -> tuple[str, str, float, int, int]:
    input_str = parse_input(input_obj[0], input_obj[1])
    start = time.perf_counter()
    proc = subprocess.run(
        ["./main"], input=input_str, text=True, capture_output=True
        )
    end = time.perf_counter()

    return proc.stdout, proc.stderr, end - start, len(input_obj[0]), len(input_obj[1])

In [ ]:
gpu_result = {}

counter = 0
for i in test_list:
    counter += 1
    if counter%50 == 0: print(counter)

    test_output = benchmark_input_gpu(i)
    key = str(test_output[-1]) + " " + str(test_output[-2])    
              
    gpu_result[key] = test_output

50
100
150
200
250
300
350
400
450
500
550


## Save benchmark files to json

In [ ]:
import json
with open('result_omp.json', 'w') as fp:
    json.dump(omp_result, fp)

with open('result_gpu.json', 'w') as fp:
    json.dump(gpu_result, fp)